# Assignment 7

## Submit as an HTML file

### Print your name below

In [1]:
print("Angie Siaca")

Angie Siaca


### Import the "pandas" "numpy" and "statsmodels.formula.api" libraries

In [2]:
# Write your answer here:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf



#### In the code chunk below read the CSV file named `results.csv` in the `data` <br> folder and print the first 5 rows of the dataset. Browse the dataset.

In [ ]:

results = pd.read_csv('data/results.csv')
print(results.head())


   resultId  raceId  driverId  constructorId number  grid position  \
0         1      18         1              1     22     1        1   
1         2      18         2              2      3     5        2   
2         3      18         3              3      7     7        3   
3         4      18         4              4      5    11        4   
4         5      18         5              1     23     3        5   

  positionText  positionOrder  points  laps         time milliseconds  \
0            1              1    10.0    58  1:34:50.616      5690616   
1            2              2     8.0    58       +5.478      5696094   
2            3              3     6.0    58       +8.163      5698779   
3            4              4     5.0    58      +17.181      5707797   
4            5              5     4.0    58      +18.014      5708630   

  fastestLap rank fastestLapTime fastestLapSpeed  statusId  
0         39    2       1:27.452         218.300         1  
1         41    3 

### (a)  Check Column Types and Data Cleaning

- Use the function .dtypes to get the column types
- Identify which columns have data types that might need conversion
- The 'milliseconds' column contains string values that should be numeric. Create a new column called 'race_time_ms' that:
    - Converts the column to a numeric data type
    - Replaces any non-numeric values with NaN

In [ ]:
# Write your answer here

print("Column Data Types:\n", results.dtypes)


potential_issues = results.select_dtypes(include='object').columns
print("\nColumns that might need conversion:", list(potential_issues))

if 'milliseconds' in results.columns:
    results['race_time_ms'] = pd.to_numeric(results['milliseconds'], errors='coerce')


print("\nSample data with cleaned 'race_time_ms' column:")
print(results[['milliseconds', 'race_time_ms']].head())



Column Data Types:
 resultId             int64
raceId               int64
driverId             int64
constructorId        int64
number              object
grid                 int64
position            object
positionText        object
positionOrder        int64
points             float64
laps                 int64
time                object
milliseconds        object
fastestLap          object
rank                object
fastestLapTime      object
fastestLapSpeed     object
statusId             int64
dtype: object

Columns that might need conversion: ['number', 'position', 'positionText', 'time', 'milliseconds', 'fastestLap', 'rank', 'fastestLapTime', 'fastestLapSpeed']

Sample data with cleaned 'race_time_ms' column:
  milliseconds  race_time_ms
0      5690616     5690616.0
1      5696094     5696094.0
2      5698779     5698779.0
3      5707797     5707797.0
4      5708630     5708630.0


### (b) Create Categorical Variables

- Create a new column called 'finish_category' that categorizes the race finish positions as follows:
    - Positions 1-3: 'Podium'
    - Positions 4-10: 'Points'
    - Positions 11-20: 'Midfield'
    - Positions >20: 'Backmarker'

Hint: Use the pd.cut() function

In [ ]:
# Write your answer here
bins = [0, 3, 10, 20, float('inf')]
labels = ['Podium', 'Points', 'Midfield', 'Backmarker']

results['finish_category'] = pd.cut(results['positionOrder'], bins=bins, labels=labels)

print(results[['positionOrder', 'finish_category']].head())


   positionOrder finish_category
0              1          Podium
1              2          Podium
2              3          Podium
3              4          Points
4              5          Points


### (c) Calculate Race Duration
- For rows where 'milliseconds' is available, create a new column <br>
'race_duration_minutes' that converts milliseconds to minutes by dividing <br>
by (1000*60).
- Display the average race duration by 'constructorId' for the top 5 <br>
constructors with the shortest average race times

In [ ]:
results['milliseconds'] = pd.to_numeric(results['milliseconds'], errors='coerce')
results['race_duration_minutes'] = results['milliseconds'] / (1000 * 60)

avg_race_constructor = results.groupby('constructorId')['race_duration_minutes'].mean()

top_5_shortest_avg = avg_race_constructor.nsmallest(5)

print(top_5_shortest_avg)

constructorId
35    76.710777
29    77.604125
41    87.046767
16    89.428828
53    89.658852
Name: race_duration_minutes, dtype: float64


### (d) Driver Performance Analysis

- Calculate the following statistics for each driver, grouped by 'driverId':
    - Average finishing position
    - Total points
    - Number of races completed
    - Best finishing position

- Sort the results by total points in descending order
- Display the top 10 drivers based on total points

In [ ]:
results['position'] = pd.to_numeric(results['position'], errors='coerce')
results['points'] = pd.to_numeric(results['points'], errors='coerce')

driver_stats = results.groupby('driverId').agg(
    average_position=('position', 'mean'),
    total_points=('points', 'sum'),
    races_completed=('driverId', 'count'),
    best_position=('position', 'min')
)

driver_stats_sorted = driver_stats.sort_values(by='total_points', ascending=False)

top_10_drivers = driver_stats_sorted.head(10)

print(top_10_drivers)


          average_position  total_points  races_completed  best_position
driverId                                                                
1                 3.383803        4396.5              310            1.0
20                5.282443        3098.0              300            1.0
4                 6.257732        2061.0              358            1.0
830               4.022388        1983.5              163            1.0
8                 6.238596        1873.0              352            1.0
822               6.296089        1778.0              201            1.0
3                 6.355932        1594.5              206            1.0
30                3.701245        1566.0              308            1.0
817               8.261538        1307.0              232            1.0
18                7.409091        1235.0              309            1.0


### (e) Linear Regression
Create a linear regression model that predicts 'points' based on 'grid' (starting position) and 'laps' completed <br>
Use the following steps:

- Clean the data to remove any non-numeric values and missing values
- Create the regression formula using smf.ols 
- Display the summary of the regression model using model.summary()

What is the predicted points for a driver starting in position 3 and completing 55 laps?

Hint: Use ```.dropna()''' to remove missing values from the points, grid, and laps <br>
variables.

In [15]:
# Write your answer here
results_cleaned = results[['points', 'grid', 'laps']].apply(pd.to_numeric, errors='coerce').dropna()
formula = 'points ~ grid + laps'
model = smf.ols(formula=formula, data=results_cleaned).fit()
print(model.summary())
predicted_points = model.predict(pd.DataFrame({'grid': [3], 'laps': [55]}))
print(f"Predicted points for a driver starting in position 3 and completing 55 laps: {predicted_points[0]}")



                            OLS Regression Results                            
Dep. Variable:                 points   R-squared:                       0.215
Model:                            OLS   Adj. R-squared:                  0.215
Method:                 Least Squares   F-statistic:                     3530.
Date:                Mon, 24 Mar 2025   Prob (F-statistic):               0.00
Time:                        21:47:05   Log-Likelihood:                -70440.
No. Observations:               25840   AIC:                         1.409e+05
Df Residuals:                   25837   BIC:                         1.409e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.5841      0.054     48.267      0.0